In [1]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext
sc=SparkContext()

In [4]:
import json

In [5]:
rdd=sc.textFile("/home/deepika/Downloads/netflix_titles.json")

In [6]:
data= rdd.map(lambda x: json.loads(x))

In [7]:
from pyspark.storagelevel import StorageLevel
data.persist(StorageLevel.MEMORY_ONLY)

In [8]:
data.take(1)

[{'show_id': 's1',
  'type': 'TV Show',
  'title': '3%',
  'director': '',
  'cast': 'João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi',
  'country': 'Brazil',
  'date_added': 'August 14, 2020',
  'release_year': '2020',
  'rating': 'TV-MA',
  'duration': '4 Seasons',
  'listed_in': 'International TV Shows, TV Dramas, TV Sci-Fi & Fantasy',
  'description': 'In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor.'}]


# how many titles actor/actress appeared in?

In [ ]:
titles= data.flatMap(lambda x: (x['cast'].split(",")))

In [ ]:
gap=titles.filter(lambda x: x !='')

In [ ]:
result= gap.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)

In [ ]:
result.collect()

In [ ]:
import os
os.environ["PYTHONHASHSEED"]=str(232)

In [ ]:
result.lookup(' Jim Sturgess')

# How many movies were released in 2008?

In [ ]:
movies=data.filter(lambda x:x['release_year']=='2008')

In [ ]:
movies.count()

In [ ]:
movies.collect()

# List all the movies whose duration is greater than 100 mins ?

In [27]:
pairs = data.map(lambda x: (int(x['duration'].split(" ")[0]), x['title']))

In [28]:
pairs.collect()

[(4, '3%'),
 (93, '7:19'),
 (78, '23:59'),
 (80, '9'),
 (123, '21'),
 (1, '46'),
 (95, '122'),
 (119, '187'),
 (118, '706'),
 (143, '1920'),
 (103, '1922'),
 (1, '1983'),
 (1, '1994'),
 (89, '2,215'),
 (91, '3022'),
 (149, 'Oct-01'),
 (1, 'Feb-09'),
 (144, '22-Jul'),
 (124, '15-Aug'),
 (87, "'89"),
 (110, '\u200b\u200bKuch Bheege Alfaaz'),
 (128, '\u200bGoli Soda 2'),
 (117, '\u200bMaj Rati \u200b\u200bKeteki'),
 (100, '\u200bMayurakshi'),
 (2, '\u200bSAINT SEIYA: Knights of the Zodiac'),
 (84, '(T)ERROR'),
 (1, '(Un)Well'),
 (99, '#Alive'),
 (95, '#AnneFrank - Parallel Stories'),
 (1, '#blackAF'),
 (90, '#cats_the_mewvie'),
 (102, '#FriendButMarried'),
 (104, '#FriendButMarried 2'),
 (99, '#realityhigh'),
 (105, '#Roxy'),
 (56, '#Rucker50'),
 (125, '#Selfie'),
 (119, '#Selfie 69'),
 (1, 'แผนร้ายนายเจ้าเล่ห์'),
 (81, '¡Ay, mi madre!'),
 (97, 'Çarsi Pazar'),
 (106, 'Ég man þig'),
 (99, 'Çok Filim Hareketler Bunlar'),
 (107, 'Òlòtūré'),
 (93, 'Æon Flux'),
 (1, 'Şubat'),
 (89, '1 Chance 2

In [43]:
duration_above_hundred= pairs.filter(lambda x: x[0] >100).groupByKey().mapValues(list).filter(lambda x: x[0]!='')

In [46]:
duration_above_hundred.collect()

[(118,
  ['706',
   'Ant-Man and the Wasp',
   'Bathinda Express',
   'Bright',
   'Chalay Thay Saath',
   'Chupan Chupai',
   'Colkatay Columbus',
   'Daagdi Chaawl',
   'Dilan 1991',
   'Dolemite Is My Name',
   'Fear and Loathing in Las Vegas',
   'Fierce',
   'Filosofi Kopi The Movie',
   'Four Weddings and a Funeral',
   'Gelo',
   'Helios',
   'Heneral Luna',
   'High & Low The Movie 3 / Final Mission',
   'History of Joy',
   'Hitch',
   'Hokkabaz',
   'Incredibles 2 (Spanish Version)',
   'Kartini: Princess of Java',
   'Kate & Leopold',
   'Lagos Real Fake Life',
   'Liefling',
   "Logan's Run",
   'Macchli Jal Ki Rani Hai',
   'Memoir of a Murderer',
   'Nightcrawler',
   'Oye Lucky! Lucky Oye!',
   'Phir Se',
   'Revelations',
   "River's Edge",
   'Room',
   'Saath Saath',
   'Saavat',
   'Saheb Bibi Golaam',
   'The Black Godfather',
   'The Book of Eli',
   'The Breakup Playlist',
   "The Butterfly's Dream",
   'The Command',
   'The Incredibles 2',
   "The King's Speech"

In [44]:
duration_above_hundred.count()

107

# List movies played by “Kareena Kapoor” ?

In [9]:
kareena = data.map(lambda x: (x['cast'].split(",")[0], x['title']))

In [10]:
kareena.collect()

[('João Miguel', '3%'),
 ('Demián Bichir', '7:19'),
 ('Tedd Chan', '23:59'),
 ('Elijah Wood', '9'),
 ('Jim Sturgess', '21'),
 ('Erdal Beşikçioğlu', '46'),
 ('Amina Khalil', '122'),
 ('Samuel L. Jackson', '187'),
 ('Divya Dutta', '706'),
 ('Rajneesh Duggal', '1920'),
 ('Thomas Jane', '1922'),
 ('Robert Więckiewicz', '1983'),
 ('', '1994'),
 ('Artiwara Kongmalai', '2,215'),
 ('Omar Epps', '3022'),
 ('Sadiq Daba', 'Oct-01'),
 ('Shahd El Yaseen', 'Feb-09'),
 ('Anders Danielsen Lie', '22-Jul'),
 ('Rahul Pethe', '15-Aug'),
 ('Lee Dixon', "'89"),
 ('Geetanjali Thapa', '\u200b\u200bKuch Bheege Alfaaz'),
 ('Samuthirakani', '\u200bGoli Soda 2'),
 ('Adil Hussain', '\u200bMaj Rati \u200b\u200bKeteki'),
 ('Soumitra Chatterjee', '\u200bMayurakshi'),
 ('Bryson Baugus', '\u200bSAINT SEIYA: Knights of the Zodiac'),
 ('', '(T)ERROR'),
 ('', '(Un)Well'),
 ('Yoo Ah-in', '#Alive'),
 ('Helen Mirren', '#AnneFrank - Parallel Stories'),
 ('Kenya Barris', '#blackAF'),
 ('', '#cats_the_mewvie'),
 ('Adipati Dolke

In [36]:
kareena.groupByKey().mapValues(list).filter(lambda x: x[0]!='').collect()

[('Tedd Chan', ['23:59']),
 ('Jim Sturgess', ['21', 'Kidnapping Mr. Heineken']),
 ('Amina Khalil', ['122']),
 ('Omar Epps', ['3022']),
 ('Sadiq Daba', ['Oct-01']),
 ('Anders Danielsen Lie', ['22-Jul']),
 ('Rahul Pethe', ['15-Aug']),
 ('Soumitra Chatterjee', ['\u200bMayurakshi', 'Peace Haven']),
 ('Yoo Ah-in', ['#Alive', 'Burning', 'Chicago Typewriter']),
 ('Helen Mirren',
  ['#AnneFrank - Parallel Stories',
   "Legend of the Guardians: The Owls of Ga'Hoole",
   'The Hundred-Foot Journey',
   'Winchester']),
 ('Adipati Dolken',
  ['#FriendButMarried',
   '#FriendButMarried 2',
   'Hunter in the Blue Side of Manchester']),
 ('Flavia Hojda', ['#Selfie']),
 ('Maia Morgenstern', ['#Selfie 69']),
 ('Estefanía de los Santos', ['¡Ay, mi madre!']),
 ('Erdem Yener', ['Çarsi Pazar']),
 ('Charlize Theron',
  ['Æon Flux', 'Dark Places', 'The Old Guard', 'Young Adult']),
 ('Lexi Giovagnoli', ['1 Chance 2 Dance', "All Hallows' Eve"]),
 ('Steven Strait', ['10,000 B.C.']),
 ('Doh Kyung-soo', ['100 Days

In [11]:
kareena1=[]
kareenaa= kareena.groupByKey().map(lambda x : (x[0], list(x[1])))

In [12]:
kareenaa.collect()

[('Tedd Chan', ['23:59']),
 ('Jim Sturgess', ['21', 'Kidnapping Mr. Heineken']),
 ('Amina Khalil', ['122']),
 ('',
  ['1994',
   '(T)ERROR',
   '(Un)Well',
   '#cats_the_mewvie',
   '#Rucker50',
   '100 Days Of Solitude',
   "100 Years: One Woman's Fight for Justice",
   '13TH',
   '14 Minutes from Earth',
   '3 Seconds Divorce',
   '60 Days In',
   '7 Days Out',
   '72 Cutest Animals',
   'A 3 Minute Hug',
   'A Gray State',
   'A Leaf of Faith',
   'A Life of Speed: The Juan Manuel Fangio Story',
   'A Lion in the House',
   'A Love Song for Latasha',
   'A Murder in the Park',
   'A new Capitalism',
   'A Perfect Crime',
   'A Secret Love',
   'A Tale of Two Kitchens',
   'A Week in Watts',
   'Abducted in Plain Sight',
   'Adhugo',
   'Afflicted',
   'After Maria',
   'After Porn Ends',
   'After Porn Ends 3',
   'After the Raid',
   'Age of Tanks',
   'Ajaibnya Cinta',
   'Alarmoty in the Land of Fire',
   'Alias JJ, la celebridad del mal',
   'Alien Worlds',
   'Alive and Kicking

In [14]:
movie_list_by_kareena_kapoor=kareenaa.lookup("Kareena Kapoor")

In [15]:
movie_list_by_kareena_kapoor

[['Chameli', 'Ek Main Aur Ekk Tu', 'Heroine', 'Kurbaan', 'Satyagraha']]

In [ ]:
movie_list_by_kareena_kapoor.saveAsTextFile("/home/deepika/my_project_dir/my_project_env/movie_list_by_kareena_kapoor/")

In [ ]:
duration_above_hundred.saveAsTextFile("/home/deepika/my_project_dir/my_project_env/duration_above_hundred/")

In [ ]:
movies.saveAsTextFile("/home/deepika/my_project_dir/my_project_env/movies_in_2008/")

In [ ]:
result.saveAsTextFile("/home/deepika/my_project_dir/my_project_env/titles_actor_actress_appeared/")